# Model 1 - grid approximation

This notebook outlines how to use model 1 as described in the paper Vincent (2015). The paper should drive one's reading, this notebooks is here to supplement one's understanding and to bridge the gap between mathematical description and algorithmic implementation.

The generative capability of the model (Figure 6, left) can be used to both generate example data, but also to generate predictions. We also use the model (Figure 6, right) to conduct inference to update our beliefs about our parameter given some observed data.

![Model1](img/model1.png)
Taken from Figure 6 of Vincent (2015).

### Proportion correct (Equation 2 in the paper)
From the paper, the proportion of correct responses is given as 

$PC = \Phi(\frac{\Delta_mu_c}{\sqrt{2\sigma^2}})$

and we can easily define this as some Julia functions

In [1]:
using StatsFuns

# define Φ() as the cumulative normal distribution function
Φ(x) = normcdf.(0, 1, x)

# define the deterministic node PC, corresponds to Equation 2 in Vincent (2015)
PC(Δμ, σ²::Number) = Φ(Δμ/sqrt(2σ²));

### Generative model (Equation 3 in the paper)
The number of correct responses $k$ will be Binomially distributed, so the generative model for this task is

$k \sim \mathrm{Binomial}(PC_c,T)$

We can define a function `generative` which will generate random samples from $P(k|T,PC)$. Note that we define two versions of this function depending upon whether the inputs are scalar values or not.

### Define a prior over our parameter (Equation 4 from the paper)

In [2]:
log_prior(σ²) = log(1/1000)

log_prior (generic function with 1 method)

In [3]:
using Distributions

function generative(T::Int64, Δμ, σ²)
    # sample from P(k|T,PC), or equivalently P(k|T,Δμ,σ²)
    k = [rand( Binomial(T,p)) for p ∈ PC(Δμ, σ²)]
end

function generative(T::Int64, Δμ::Number, σ²::Number)
    # sample from P(k|T,PC), or equivalently P(k|T,Δμ,σ²)
    k = rand(Binomial(T,PC(Δμ, σ²)))
end

generative (generic function with 2 methods)

### Define the likelihood and joint distribution (Equation 5 in the paper)
First we define the likelihood (part of Equation 5) however we do this as the sum of log probabilities.

In [4]:
function log_likelihood(σ²::Number, Δμ::Array{Float64,1}, kvals::Array{Int64,1}, T::Integer)
    log_likelihood = sum(binomlogpdf.(T, PC(Δμ, σ²), kvals))
end

log_likelihood (generic function with 1 method)

Then simply define the log posterior as the sum of the log prior and log likelihood. This allows us to compute $P(\sigma^2, \Delta\mu, k, T)$

In [5]:
log_posterior(σ², Δμ, k, T::Integer) = log_prior(σ²) + log_likelihood(σ², Δμ, k, T)

log_posterior (generic function with 1 method)

For plotting the posterior, we'll need to convert back from log density, to a normalised posterior density, so below we define this utility function.

In [6]:
function logpost2post(log_posterior_density::Array{Float64,1})
    posterior_density = exp.(log_posterior_density)
    return posterior_density ./ sum(posterior_density)
end

logpost2post (generic function with 1 method)

## STEP 1: define data
If we wanted to generate example data from this observer we can simple sample from $P(k|T,\Delta\mu,\sigma^2)$ using the `generative` function defined above. The example below shows how you would do this for $T=100$ and a set of $\Delta\mu$ values, with $\sigma^2=1$.

    generative(100, [0.0100, 0.0215, 0.0464, 0.1000, 0.2154, 0.4642, 1.0000, 2.1544, 4.6416, 10.0000], 1)
    
If you try runing this code a number of times, you'll see that we get a different set of possible experimental results $k$ each time we sample from $P(k|T,\Delta\mu,\sigma^2)$.

In [7]:
generative(100, [0.0100, 0.0215, 0.0464, 0.1000, 0.2154, 0.4642, 1.0000, 2.1544, 4.6416, 10.0000], 1)

10-element Array{Int64,1}:
  56
  55
  50
  48
  60
  59
  82
  96
 100
 100

But for this example, we will define our own observed data where:
- `Δμ` = stimulus intensities
- `k` = number of correct responses for each stimulus intensity (could range from 0 to `T`)
- `T` = number of trials per stimulus intensity, 100 in this example

Proportion correct (per stimulus level) is given by `k./T`. We'll define this data in a Dictionary

In [8]:
data = Dict()
data["T"] = 100
data["k"] = [50, 51, 57, 55, 63, 62, 82, 94, 99, 100]
data["Δμ"] = [0.0100, 0.0215, 0.0464, 0.1000, 0.2154, 0.4642, 1.0000, 2.1544, 4.6416, 10.0000];

And we can now visualise this data in a plot (see Figure 8 in the paper)

In [9]:
using Plots

plot_data_space = scatter(log.(data["Δμ"]), data["k"]./data["T"],
    xlabel = "log signal instensity, Δμ", #xscale = :log,
    ylabel = "proportion correct, k/T",
    legend = false)

# TODO: xaxis should be log scale
# TODO: get latex axis labels

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 log signal instensity, ?? 
 
 
 proportion correct, k/T

## STEP 2: parameter recovery
Evaluate posterior over range of σ2 values. We are using grid approximation here, so we will define a fine grid of many σ2 values.

In [10]:
σ² = linspace(0.5, 3, 10.0^3)

0.5:0.0025025025025025025:3.0

### Do grid approximation
This is really simple, all we are doing is iterating over a vector of σ2 values, evaluating the log posterior for each

In [11]:
function gridApprox(σ²_vector, data)
    σ²_log_posterior_density = [log_posterior(σ², data["Δμ"], data["k"], data["T"]) for σ² ∈ σ²_vector]
end

σ²_log_posterior_density = gridApprox(σ², data)
σ²_posterior_density = logpost2post(σ²_log_posterior_density);

### Plot our posterior over the parameter (see Figure 8 in the paper)

In [12]:
plot_parameter_space = plot(σ², σ²_posterior_density,
    xlabel = "sigma^2",
    ylabel = "posterior density",
    legend=false)
# TODO: get latex axis labels

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 0.001 
 
 
 0.002 
 
 
 0.003 
 
 
 0.004 
 
 
 sigma^2 
 
 
 posterior density 
 
<polyline clip-path="url(#clip7402)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 61.1494,359.994 61.6691,359.984 62.1888,359.973 62.7085,359.96 63.2281,359.946 63.7478,359.929 64.2675,359.912 64.7872,359.892 65.3069,359.869 65.8266,359.845 
 66.3462,359.818 66.8659,359.787 67.3856,359.754 67.9053,359.717 68.425,359.677 68.9447,359.632 69.4644,359.583 69.984,359.53 70.5037,359.471 71.0234,359.407 
 71.5431,359.337 72.0628,359.261 72.5825,359.178 73.1022,359.088 73.6218,358.99 74.1415,358.884 74.6612,358.769 75.1809,358.645 75.7006,358.511 76.2203,358.366 
 76.74,358.211 77.2596,358.043 77.7793,357.863 78.299,357.67 78.8187,357.463 79.3384,357.241 79.8581,357.004 80.3778,356.75 80.8974,356.48 81.4171,356.192 
 81.9368,355.885 82.4565,355.559 82.9762,355.212 83.4959,354.844 84.0155,354.453 84.5352,354.04 85.0549,353.602 85.5746,353.14 86.0943,352.651 86.614,352.136 
 87.1337,351.592 87.6533,351.02 88.173,350.418 88.6927,349.785 89.2124,349.121 89.7321,348.423 90.2518,347.692 90.7715,346.927 91.2911,346.125 91.8108,345.288 
 92.3305,344.412 92.8502,343.498 93.3699,342.545 93.8896,341.552 94.4093,340.518 94.9289,339.442 95.4486,338.323 95.9683,337.161 96.488,335.955 97.0077,334.703 
 97.5274,333.406 98.047,332.063 98.5667,330.673 99.0864,329.235 99.6061,327.75 100.126,326.215 100.645,324.632 101.165,322.999 101.685,321.317 102.205,319.584 
 102.724,317.801 103.244,315.967 103.764,314.083 104.283,312.147 104.803,310.16 105.323,308.123 105.842,306.034 106.362,303.894 106.882,301.704 107.401,299.463 
 107.921,297.171 108.441,294.83 108.96,292.439 109.48,289.998 110,287.509 110.519,284.972 111.039,282.387 111.559,279.755 112.079,277.076 112.598,274.352 
 113.118,271.584 113.638,268.771 114.157,265.916 114.677,263.018 115.197,260.08 115.716,257.102 116.236,254.085 116.756,251.03 117.275,247.939 117.795,244.813 
 118.315,241.653 118.834,238.46 119.354,235.237 119.874,231.984 120.394,228.702 120.913,225.394 121.433,222.061 121.953,218.704 122.472,215.325 122.992,211.925 
 123.512,208.507 124.031,205.072 124.551,201.621 125.071,198.156 125.59,194.68 126.11,191.193 126.63,187.698 127.149,184.196 127.669,180.689 128.189,177.179 
 128.708,173.668 129.228,170.157 129.748,166.649 130.268,163.144 130.787,159.645 131.307,156.154 131.827,152.673 132.346,149.202 132.866,145.745 133.386,142.303 
 133.905,138.877 134.425,135.469 134.945,132.081 135.464,128.715 135.984,125.372 136.504,122.055 137.023,118.764 137.543,115.501 138.063,112.268 138.582,109.067 
 139.102,105.898 139.622,102.764 140.142,99.6666 140.661,96.6061 141.181,93.5844 141.701,90.603 142.22,87.6631 142.74,84.7661 143.26,81.9132 143.779,79.1057 
 144.299,76.3447 144.819,73.6316 145.338,70.9672 145.858,68.3528 146.378,65.7894 146.897,63.278 147.417,60.8195 147.937,58.4149 148.457,56.065 148.976,53.7707 
 149.496,51.5328 150.016,49.352 150.535,47.229 151.055,45.1645 151.575,43.1591 152.094,41.2134 152.614,39.328 153.134,37.5033 153.653,35.7397 154.173,34.0378 
 154.693,32.3979 155.212,30.8202 155.732,29.3052 156.252,27.853 156.771,26.4638 157.291,25.1379 157.811,23.8753 158.331,22.6762 158.85,21.5406 159.37,20.4685 
 159.89,19.4599 160.409,18.5147 160.929,17.633 161.449,16.8144 161.968,16.0589 162.488,15.3663 163.008,14.7364 163.527,14.1688 164.047,13.6634 164.567,13.2197 
 165.086,12.8375 165.606,12.5163 166.126,12.2558 166.646,12.0554 167.165,11.9149 167.685,11.8336 168.205,11.811 168.724,11.8467 169.244,11.94 169.764,12.0904 
 170.283,12.2973 170.803,12.5601 171.323,12.878 171.842,13.2505 172.362,13.6769 172.882,14.1564 173.401,14.6884 173.921,15.2721 174.441,15.9068 174.96,16.5917 
 175.48,17.326 176,18.1091 176.52,18.9399 177.039,19.8179 177.559,20.742 178.079,21.7116 178.598,22.7257 179.1

## STEP 3: Predictive distribution
Now we have our posterior over parameters we can use the generative ability of the model to sample predicted behaviour $k$ given our knowledge of $\sigma^2$. Below, we generate a set of predictions for many interpolated signal intensity levels (`Δμ_interp_vals`) for which we do not have data for. This is a useful demonstration of making and visualising the model's predictions.

In [13]:
Δμ_interp_vals = logspace(-2, 1, 20);

Draw many samples from the posterior distribution of internal variance

In [14]:
using StatsBase # for sampling
nSamples = 10^2
σ²_samples = sample(σ², ProbabilityWeights(σ²_posterior_density), nSamples)

100-element Array{Float64,1}:
 0.735235
 1.06306 
 1.45095 
 1.40841 
 0.882883
 0.917918
 1.02052 
 1.08058 
 1.32332 
 1.48849 
 1.23323 
 1.07808 
 0.94044 
 ⋮       
 1.32082 
 1.1957  
 1.21572 
 1.37838 
 0.945445
 0.867868
 1.06557 
 0.79029 
 0.755255
 1.003   
 1.40591 
 1.1031  

Now we will generate many predicted data points $k$ over each interpolated Δμ values, with σ2 sampled from our posterior 

In [15]:
# do the posterior prediction
predk = [generative(data["T"], Δμ, σ²) for Δμ ∈ Δμ_interp_vals, σ² ∈ σ²_samples]

20×100 Array{Int64,2}:
  49   46   53   49   36   57   56   45  …   42   45   42   45   44   49   53
  58   47   53   45   52   43   49   55      52   57   55   54   49   52   56
  56   46   47   61   40   53   48   51      65   52   42   37   59   50   48
  53   49   50   56   54   48   45   53      62   57   55   57   49   53   49
  52   46   43   48   48   48   50   53      50   53   51   62   48   49   53
  50   55   62   48   48   44   45   52  …   47   54   54   53   44   52   51
  44   50   50   55   47   54   60   48      49   60   52   48   44   53   49
  57   55   47   62   59   61   47   52      50   56   56   53   49   55   51
  54   56   61   50   53   51   56   52      58   57   51   48   47   61   52
  57   66   59   56   52   57   62   63      49   54   52   57   53   58   58
  66   65   54   61   64   65   64   66  …   49   57   66   63   58   58   62
  63   67   66   60   66   64   69   64      65   72   61   71   69   63   62
  80   72   70   70   77   77   80   66  

# ---- CODE BELOW IS IN PROGRESS ----

In [16]:
# # Calculate 95% CI's for each signal level
# #CI = prctile(predk,[5 95]) ./ data.T;

Save

In [17]:
# save('output/m1MAPestimate.mat', 'vMode')
# save(['~/Dropbox/tempModelOutputs/tempModel1run_gridApprox.mat'], '-v7.3')

In [18]:
plot(plot_data_space)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 log signal instensity, ?? 
 
 
 proportion correct, k/T

Overall results should look something like this
![Model1](img/model1results.png)
Taken from Figure 8 of Vincent (2015).

## References
Vincent, B. T. (2015). A tutorial on Bayesian models of perception. Journal of Mathematical Psychology, 66, 103–114. http://doi.org/10.1016/j.jmp.2015.02.001